## Importing the necessary libraries

In [1]:
import os
import cv2
import numpy as np
import glob
import imutils
from keras.preprocessing.image import img_to_array,load_img
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
os.chdir('C:\\Users\\win10')
owd = os.getcwd()
print(owd)
os.chdir(owd)

C:\Users\win10


## Setting directory path variable and Function to process and store video frames

In [2]:
store_image=[]
train_path='C:\\Projects\\Video Surveilance\\Train path'
fps=5
train_videos=os.listdir('C:\\Projects\\Video Surveilance\\Avenue_Dataset\\Avenue Dataset\\training_videos')
print(train_videos)
train_images_path=train_path+'/frames'
os.makedirs(train_images_path)


['01.avi', '01.mp4', '02.avi', '02.mp4', '03.avi', '03.mp4', '04.avi', '04.mp4', '05.avi', '05.mp4', '06.avi', '06.mp4', '07.avi', '07.mp4', '08.avi', '08.mp4', '09.avi', '09.mp4', '10.avi', '10.mp4', '11.avi', '11.mp4', '12.avi', '12.mp4', '13.avi', '13.mp4', '14.avi', '14.mp4', '15.avi', '15.mp4', '16.avi', '16.mp4', '17.avi', '18.avi', '19.avi', '20.avi', '21.avi']


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Projects\\Video Surveilance\\Train path/frames'

## Extract frames from video and call store function

In [40]:
print(train_videos)
os.chdir(train_images_path)
#for video in train_videos:
    #print(video)
vidcap = cv2.VideoCapture('C:\\Projects\\Video Surveilance\\Avenue_Dataset\\Avenue Dataset\\training_videos\\16.mp4')
print(vidcap)
success,image = vidcap.read()
count=0;
while success:
    cv2.imwrite("Vid16-%d.jpg" % count, image)
    success,image = vidcap.read()
    count += 1

['01.avi', '01.mp4', '02.avi', '02.mp4', '03.avi', '03.mp4', '04.avi', '04.mp4', '05.avi', '05.mp4', '06.avi', '06.mp4', '07.avi', '07.mp4', '08.avi', '08.mp4', '09.avi', '09.mp4', '10.avi', '10.mp4', '11.avi', '11.mp4', '12.avi', '12.mp4', '13.avi', '13.mp4', '14.avi', '14.mp4', '15.avi', '15.mp4', '16.avi', '16.mp4', '17.avi', '18.avi', '19.avi', '20.avi', '21.avi']
<VideoCapture 000001B1373E9CF0>


## Store the store_image list in a numpy file

In [ ]:
os.chdir(train_path)

def store_inarray(image_path):
    image=load_img(image_path)
    image=img_to_array(image)
    image=cv2.resize(image, (227,227), interpolation = cv2.INTER_AREA)
    gray=0.2989*image[:,:,0]+0.5870*image[:,:,1]+0.1140*image[:,:,2]
    store_image.append(gray)

images=os.listdir(train_images_path)
for image in images:
    image_path=train_images_path + '/' + image
    store_inarray(image_path)

store_image=np.array(store_image)
a,b,c = store_image.shape
print(a,b,c)

In [ ]:
store_image.resize(b,c,a)
store_image=(store_image-store_image.mean())/(store_image.std())
store_image=np.clip(store_image,0,1)
np.save('training.npy',store_image)

In [ ]:
stae_model=Sequential()
stae_model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
stae_model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))
stae_model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))
stae_model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [ ]:
training_data=np.load('training.npy')
frames=training_data.shape[2]
frames=frames-frames%10
training_data=training_data[:,:,:frames]
training_data=training_data.reshape(-1,227,227,10)
training_data=np.expand_dims(training_data,axis=4)
target_data=training_data.copy()
epochs=5
batch_size=1
callback_save = ModelCheckpoint("saved_model.h5", monitor="mean_squared_error", save_best_only=True)
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)
stae_model.fit(training_data,target_data, batch_size=batch_size, epochs=epochs, callbacks = [callback_save,callback_early_stopping])
stae_model.save("saved_model.h5")

# TESTING

In [ ]:
import cv2
import numpy as np 
from keras.models import load_model
import argparse
from PIL import Image
import imutils

In [ ]:
def mean_squared_loss(x1,x2):
    difference=x1-x2
    a,b,c,d,e=difference.shape
    n_samples=a*b*c*d*e
    sq_difference=difference**2
    Sum=sq_difference.sum()
    distance=np.sqrt(Sum)
    mean_distance=distance/n_samples
    return mean_distance

In [ ]:
model=load_model("saved_model.h5")
cap = cv2.VideoCapture("C:\\Projects\\Video Surveilance\\Avenue_Dataset\\Avenue Dataset\\testing_videos\\03.avi")
print(cap.isOpened())

In [ ]:
while cap.isOpened():
    imagedump=[]
    ret,frame=cap.read()
    for i in range(10):
        ret,frame=cap.read()
        image = imutils.resize(frame,width=700,height=600)
        frame=cv2.resize(frame, (227,227), interpolation = cv2.INTER_AREA)
        gray=0.2989*frame[:,:,0]+0.5870*frame[:,:,1]+0.1140*frame[:,:,2]
        gray=(gray-gray.mean())/gray.std()
        gray=np.clip(gray,0,1)
        imagedump.append(gray)
    imagedump=np.array(imagedump)
    imagedump.resize(227,227,10)
    imagedump=np.expand_dims(imagedump,axis=0)
    imagedump=np.expand_dims(imagedump,axis=4)
    output=model.predict(imagedump)
    loss=mean_squared_loss(imagedump,output)
    if frame.any()==None:
        print("none")
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
    if loss>0.00068:
        print('Abnormal Event Detected')
        cv2.putText(image,"Abnormal Event",(100,80),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)
    cv2.imshow("video",image)
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np 
from keras.models import load_model
import argparse
from PIL import Image
import imutils


def mean_squared_loss(x1,x2):
    difference=x1-x2
    a,b,c,d,e=difference.shape
    n_samples=a*b*c*d*e
    sq_difference=difference**2
    Sum=sq_difference.sum()
    distance=np.sqrt(Sum)
    mean_distance=distance/n_samples

    return mean_distance


model=load_model("C:\\Projects\\Video Surveilance\\Train path\\saved_model.h5")

cap = cv2.VideoCapture("C:\\Projects\\Video Surveilance\\Avenue_Dataset\\Avenue Dataset\\testing_videos\\03.avi")
print(cap.isOpened())

while cap.isOpened():
    imagedump=[]
    ret,frame=cap.read()

    for i in range(10):
        ret,frame=cap.read()
        image = imutils.resize(frame,width=700,height=600)

        frame=cv2.resize(frame, (227,227), interpolation = cv2.INTER_AREA)
        gray=0.2989*frame[:,:,0]+0.5870*frame[:,:,1]+0.1140*frame[:,:,2]
        gray=(gray-gray.mean())/gray.std()
        gray=np.clip(gray,0,1)
        imagedump.append(gray)

    imagedump=np.array(imagedump)

    imagedump.resize(227,227,10)
    imagedump=np.expand_dims(imagedump,axis=0)
    imagedump=np.expand_dims(imagedump,axis=4)

    output=model.predict(imagedump)

    loss=mean_squared_loss(imagedump,output)

    if frame.any()==None:
        print("none")

    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
    if loss>0.00068:
        print('Abnormal Event Detected')
        cv2.putText(image,"Abnormal Event",(100,80),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)

    cv2.imshow("video",image)

cap.release()
cv2.destroyAllWindows()